In [1]:
from collections.abc import Mapping
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

import zipfile
import pandas as pd
import json
import openai
import boto3

In [ ]:
client = Elasticsearch(IP)

In [4]:
data = {}
for key in ['title', 'artists', 'album', 'videoId', 'youtube_link', 'youtube_embed', 'album_art', 'lyrics']:
    data[key] = []

def read_file_line_by_line(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

# 사용 예시
file_path = 'songs.json'  # 실제 파일 경로로 변경하세요

for line in tqdm(read_file_line_by_line(file_path)):
    row = eval(line)
    for key in ['title', 'artists', 'album', 'videoId', 'youtube_link', 'youtube_embed', 'album_art', 'lyrics']:
        row[key] = row[key].replace("\/", "/").replace('\n', ' ')
        data[key].append(row[key])

df = pd.DataFrame(columns=['title', 'artists', 'album', 'videoId', 'youtube_link', 'youtube_embed', 'album_art', 'lyrics'], data=data)
df['LYRICS_VECTOR'] = None
df

821it [00:00, 8597.93it/s]


,title,artists,album,videoId,youtube_link,youtube_embed,album_art,lyrics,LYRICS_VECTOR
0,(Don't Fear) The Reaper,Blue Oyster Cult,Agents Of Fortune,FS8p_F0Stog,https://www.youtube.com/watch?v=FS8p_F0Stog,https://www.youtube.com/embed/FS8p_F0Stog,https://lh3.googleusercontent.com/RXkX2lDDXfpQ...,"All our times have come Here, but now they're ...",None
1,(I Just) Died In Your Arms,Cutting Crew,Broadcast,2wf-MNzSbpA,https://www.youtube.com/watch?v=2wf-MNzSbpA,https://www.youtube.com/embed/2wf-MNzSbpA,https://lh3.googleusercontent.com/xVk_y8e7_tCd...,"Oh I, I just died in your arms tonight It must...",None
2,...Baby One More Time,Britney Spears,...Baby One More Time (Digital Deluxe Version),91Niv2q4gvc,https://www.youtube.com/watch?v=91Niv2q4gvc,https://www.youtube.com/embed/91Niv2q4gvc,https://lh3.googleusercontent.com/5MIh2Dgbv0cR...,"[Intro] Oh, baby, baby Oh, baby, baby [Verse ...",None
3,1 TO 10,TWICE,TWICEcoaster : LANE1,wXlrZpTWKgM,https://www.youtube.com/watch?v=wXlrZpTWKgM,https://www.youtube.com/embed/wXlrZpTWKgM,https://lh3.googleusercontent.com/aUQLzHTx0thW...,"[트와이스 ""1 TO 10"" 가사] [Intro: Jihyo, Mina] (Bab...",None
4,10 Minutes (10 Minutes),Lee Hyori,Stylish… (Stylish…),91MtbP4HFPE,https://www.youtube.com/watch?v=91MtbP4HFPE,https://www.youtube.com/embed/91MtbP4HFPE,https://lh3.googleusercontent.com/splNBTizb_V_...,"Just want ten minutes, 내 것이 되는 시간 순진한 내숭에 속아 우...",None
...,...,...,...,...,...,...,...,...,...
816,한강 (HANGANG),Hoody (후디),HANGANG,xe1CuavWvKo,https://www.youtube.com/watch?v=xe1CuavWvKo,https://www.youtube.com/embed/xe1CuavWvKo,https://lh3.googleusercontent.com/JT2TdLiR3Mt3...,[후디 “한강” 가사] [Verse 1] 너와 함께 있는 여긴 바다보다 반짝이는 ...,None
817,한숨(Breathe),LeeHi,Seoulite,JsaW-lHELYo,https://www.youtube.com/watch?v=JsaW-lHELYo,https://www.youtube.com/embed/JsaW-lHELYo,https://lh3.googleusercontent.com/ZXtF0jtEAEh5...,"[이하이 ""한숨"" 가사] [Verse 1] 숨을 크게 쉬어봐요 당신의 가슴 양쪽이...",None
818,한잔해,박군,한잔해,_x7-BOafGWM,https://www.youtube.com/watch?v=_x7-BOafGWM,https://www.youtube.com/embed/_x7-BOafGWM,https://lh3.googleusercontent.com/-Lr8i1bjoS8L...,"[Intro] Yeah, Control 내 왔다 Simon Dominic [Ver...",None
819,홍시,Na Hoon-a,최신 골든 히트곡 모음집 (40주년 기념 특별 기획음반),OIJJglSSTVE,https://www.youtube.com/watch?v=OIJJglSSTVE,https://www.youtube.com/embed/OIJJglSSTVE,https://lh3.googleusercontent.com/bqWpET2IQWtS...,Chorus: 생각이 난다 홍시가 열리면 울 엄마가 생각이 난다 자장가 대신 젖가슴...,None


In [ ]:
bedrock = boto3.client('bedrock-runtime', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region)

def getEmbedding(text):
    request_body = json.dumps({
        "texts": [text],
        "input_type": "search_document",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings

def getQueyrEmbedding(query):
    request_body = json.dumps({
        "texts": [query],
        "input_type": "search_query",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings

In [ ]:
import pandas as pd
import numpy as np

error_cnt = 0
processed_count = 0

for idx, row in tqdm(df.iterrows()):
    try:
        lyrics = row['lyrics']
        lyrics_vector = getEmbedding(lyrics)
        df.at[idx, 'LYRICS_VECTOR'] = lyrics_vector
    except Exception as e:
        error_cnt+=1
        pass

In [ ]:
df.to_csv('df_music.csv')
df = df[df['LYRICS_VECTOR'].isna()!=True]
df = df.reset_index(drop=True)

In [ ]:
index_mapping = {
    "mappings": {
        "properties": {
            "lyrics_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            },
            "artists": {"type": "text"},
            "lyrics": {"type": "text"},
            "title": {"type": "text"},
            "youtube_embed": {"type": "keyword"},
            "vector_id": {"type": "long"}
        }
    }
}

client.indices.create(index="music_vector_index", body=index_mapping)

In [ ]:
def dataframe_to_bulk_actions(df):
    for index, row in df.iterrows():
        yield {
            "_index": 'music_vector_index',
            "_id": index,
            "_source": {
                'youtube_embed' : row["youtube_embed"],
                'title' : row["title"],
                'lyrics' : row["lyrics"],
                'lyrics_vector' : json.loads(str(row["LYRICS_VECTOR"])),
                'artists': row['artists'],
                'vector_id' : index
            }
        }

In [ ]:
start = 0
end = len(df)
batch_size = 100
for batch_start in tqdm(range(start, end, batch_size)):
    batch_end = min(batch_start + batch_size, end)
    batch_dataframe = df.iloc[batch_start:batch_end]
    actions = dataframe_to_bulk_actions(batch_dataframe)
    helpers.bulk(client, actions)

In [ ]:
df

In [ ]:
diary = "내일은 중요한 면접이 있는 날이다. 잘 할 수 있을지 걱정된다. 준비는 열심히 했지만, 자꾸 불안한 마음이 든다. 오늘 밤 잠을 잘 수 있을지 모르겠다. 긍정적인 마인드를 가지려 노력해야겠다."
diary = "오늘 직장에서 너무 화가 났다. 내가 열심히 준비한 프로젝트를 동료가 자기 것인 양 발표했다. 나의 노력을 인정받지 못하는 것 같아 너무 분하고 억울하다. 이런 부당한 일이 또 일어나지 않도록 해야겠다"
query = f"당신은 심리 분석가처럼 행동합니다.\n또한 당신은 모든 음악에 대한 해박한 지식을 가지고 있습니다.\n아래의 일기를 읽어보고 글쓴이에게 위로가 될만한, 혹은 도움이 될만한 음악을 추천해주시기 바랍니다.\n\n### 일기 ###\n{diary}"
print(query)

question_embedding = getQueyrEmbedding(query)

In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        title = hit['_source']['title']
        text = hit['_source']['text']
        pretty_output = (f"\nID: {id}\nTitle: {title}\nSummary: {text}\nScore: {score}")
        print(pretty_output)

In [ ]:
query = {
    "knn": {
        "query_vector": question_embedding,
        "k": 3,
        "field": "lyrics_vector"  # This remains the same, as it refers to the field in your index
    },
    "_source": ["lyrics", "title", "url"],
    "size": 3
}

response = client.search(
    index="music_vector_index",
    body=query
)

# pretty_response(response)

top_hit_summary = response['hits']['hits'][0]['_source']['lyrics']

[row['_source'] for row in response['hits']['hits']]